<a href="https://colab.research.google.com/github/Akshata4/data_engineering/blob/main/pyspark/PySpark_Working_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pyspark findspark

In [9]:
!java -version

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [10]:
import findspark
findspark.init()

In [11]:
%cd /content/

/content


In [12]:
from pyspark.sql import SparkSession
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.regression import LinearRegression

In [13]:
spark = SparkSession.builder.appName("MyApp").master("local[*]").getOrCreate()

In [14]:
df = spark.read.csv('/content/sample_data/netflix_titles.csv', header=True)

In [15]:
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         NULL|Septem

In [16]:
spark

In [17]:
df.rdd.getNumPartitions()

1

In [18]:
df1 = df.where(df.type=="Movie")

In [19]:
df1.show(5)

+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|            director|                cast|             country|        date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+
|     s1|Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|  90 min|       Documentaries|As her father nea...|
|     s7|Movie|My Little Pony: A...|Robert Cullen, Jo...|Vanessa Hudgens, ...|                NULL|September 24, 2021|        2021|    PG|  91 min|Children & Family...|Equestria's divid...|
|     s8|Movie|             Sankofa|        Haile 

In [20]:
df1.rdd.getNumPartitions()

1

In [21]:
df1.write.csv("/content/sample_data/output", header=True,mode="overwrite")

In [22]:
df.groupBy("type").count().show()

+-------------+-----+
|         type|count|
+-------------+-----+
|         NULL|    1|
|      TV Show| 2676|
|        Movie| 6131|
|William Wyler|    1|
+-------------+-----+



In [23]:
df.where(df.type=="William Wyler").show()

+----------------+-------------+-----+-------------+--------------+-------+----------+------------+--------------------+--------------------+---------+-----------+
|         show_id|         type|title|     director|          cast|country|date_added|release_year|              rating|            duration|listed_in|description|
+----------------+-------------+-----+-------------+--------------+-------+----------+------------+--------------------+--------------------+---------+-----------+
|Flying Fortress"|William Wyler| NULL|United States|March 31, 2017|   1944|     TV-PG|      40 min|Classic Movies, D...|This documentary ...|     NULL|       NULL|
+----------------+-------------+-----+-------------+--------------+-------+----------+------------+--------------------+--------------------+---------+-----------+



In [24]:
## EDA 1, also removed nul value and one errornous record

df_final = df.filter((df.type!="William Wyler"))
df_final = df_final.filter((df.type!="NULL"))
df_final.groupBy("type").count().show()

+-------+-----+
|   type|count|
+-------+-----+
|TV Show| 2676|
|  Movie| 6131|
+-------+-----+



In [25]:
## EDA 2 - What is the distribution of content by country?
## need to split country by ","

df.groupBy("country").count().show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|Peru, United Stat...|    1|
|India, United Kin...|    1|
|Japan, Canada, Un...|    1|
|United Kingdom, C...|    1|
|      India, Germany|    2|
|South Africa, Uni...|    1|
|              Russia|   15|
|United Kingdom, G...|    1|
|Chile, United Sta...|    1|
|South Africa, Angola|    1|
|United States, Po...|    1|
|  Philippines, Qatar|    1|
|Hong Kong, China,...|    1|
|  Germany, Sri Lanka|    1|
|Denmark, France, ...|    1|
|United States, Fr...|    1|
|United Kingdom, N...|    2|
|Australia, United...|    2|
|Brazil, France, G...|    1|
|United States, Ir...|    3|
+--------------------+-----+
only showing top 20 rows



In [26]:
df_final.show(3)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         NULL|Septem

In [27]:
## split country by ','

from pyspark.sql.functions import split
from pyspark.sql.functions import explode
from pyspark.sql.functions import trim

df_final = df_final.withColumn("country_array", split(df_final["country"],','))
df_final.show()
df_exploded = df_final.withColumn("country_exploded", explode(df_final["country_array"]))
df_exploded = df_exploded.withColumn("country_exploded", trim(df_exploded["country_exploded"]))
df_exploded.show()


+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|       country_array|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|     [United States]|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Se

In [28]:
df_exploded.groupBy("country_exploded").count().orderBy("count",ascending=False).show()

## top content producing countries are usa, india, uk

+----------------+-----+
|country_exploded|count|
+----------------+-----+
|   United States| 3676|
|           India| 1046|
|  United Kingdom|  805|
|          Canada|  445|
|          France|  392|
|           Japan|  318|
|     South Korea|  231|
|           Spain|  230|
|         Germany|  224|
|          Mexico|  169|
|           China|  162|
|       Australia|  160|
|           Egypt|  117|
|          Turkey|  113|
|       Hong Kong|  105|
|         Nigeria|  101|
|           Italy|  100|
|          Brazil|   97|
|       Argentina|   91|
|         Belgium|   90|
+----------------+-----+
only showing top 20 rows



In [29]:
df_final.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|       country_array|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|     [United States]|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Se

In [30]:
## EDA 3 - # of titles added in a year
## A lot of movies started coming on netflix post 2017 with a peak in 2019 and 2020, mat be due to COVID-19

from pyspark.sql.functions import to_date, year

df_final = df_final.withColumn("year_added", year(to_date(df_final['date_added'], "MMMM d, yyyy")))

df_final.groupBy("year_added").count().orderBy("year_added",ascending=False).show()

+----------+-----+
|year_added|count|
+----------+-----+
|      2021| 1491|
|      2020| 1872|
|      2019| 1997|
|      2018| 1623|
|      2017| 1162|
|      2016|  418|
|      2015|   72|
|      2014|   23|
|      2013|   10|
|      2012|    3|
|      2011|   13|
|      2010|    1|
|      2009|    2|
|      2008|    2|
|      NULL|  118|
+----------+-----+



In [31]:
df_final.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+---------------+----------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|  country_array|year_added|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+---------------+----------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|[United States]|      2021|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After cro

In [32]:
## Content insights
## EDA 4 - What are the most common genres (listed_in) on Netflix?
##Split listed_in by comma, explode, and count occurrences.

df_final=  df_final.withColumn("list_array", split(df_final.listed_in, ','))
list_exploded = df_final.withColumn("list_explode", explode(df_final.list_array))
list_exploded = list_exploded.withColumn("list_explode", trim(list_exploded["list_explode"]))
list_exploded.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+---------------+----------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|  country_array|year_added|          list_array|        list_explode|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+---------------+----------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|[United States]|      2021|     [Documentaries]|       Documentaries|


In [33]:
list_exploded.groupBy("list_explode").count().orderBy("count",ascending=False).show()
## top genres are - International Movies, Dramas, Comedies, International TV

+--------------------+-----+
|        list_explode|count|
+--------------------+-----+
|International Movies| 2748|
|              Dramas| 2419|
|            Comedies| 1670|
|International TV ...| 1350|
|       Documentaries|  866|
|  Action & Adventure|  857|
|           TV Dramas|  762|
|  Independent Movies|  751|
|Children & Family...|  641|
|     Romantic Movies|  616|
|         TV Comedies|  580|
|           Thrillers|  573|
|      Crime TV Shows|  469|
|            Kids' TV|  451|
|          Docuseries|  395|
|    Music & Musicals|  373|
|   Romantic TV Shows|  370|
|       Horror Movies|  357|
|     Stand-Up Comedy|  343|
|          Reality TV|  254|
+--------------------+-----+
only showing top 20 rows



In [34]:
## EDA 5 - Who are the top 10 directors and actors by number of titles?
## Split director and cast, explode, and count.

df_final.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+---------------+----------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|  country_array|year_added|          list_array|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+---------------+----------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|[United States]|      2021|     [Documentaries]|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Af

In [46]:
## top 10 directors

from pyspark.sql.functions import col

director = df_final.withColumn("director_array",split(df_final.director,','))
director = director.withColumn("director_explode", explode(director.director_array))
director = director.withColumn("director_explode", trim(director.director_explode)).filter(col("director_explode") != "") \
                           .filter(col("director_explode").isNotNull())

director.groupBy('director_explode').count().orderBy('count', ascending=False).show(10)
# director.show(5)

+-------------------+-----+
|   director_explode|count|
+-------------------+-----+
|      Rajiv Chilaka|   22|
|          Jan Suter|   21|
|        Raúl Campos|   19|
|       Marcus Raboy|   16|
|        Suhas Kadav|   16|
|          Jay Karas|   15|
|Cathy Garcia-Molina|   13|
|    Youssef Chahine|   12|
|    Martin Scorsese|   12|
|        Jay Chapman|   12|
+-------------------+-----+
only showing top 10 rows



In [47]:
## top 10 actors

from pyspark.sql.functions import col

cast = df_final.withColumn("cast_array",split(df_final.cast,','))
cast = cast.withColumn("cast_explode", explode(cast.cast_array))
cast = cast.withColumn("cast_explode", trim(cast.cast_explode)).filter(col("cast_explode") != "") \
                           .filter(col("cast_explode").isNotNull())

cast.groupBy('cast_explode').count().orderBy('count', ascending=False).show(10)

+----------------+-----+
|    cast_explode|count|
+----------------+-----+
|     Anupam Kher|   43|
|  Shah Rukh Khan|   35|
|   Julie Tejwani|   33|
|Takahiro Sakurai|   32|
|Naseeruddin Shah|   32|
|    Rupa Bhimani|   31|
|    Akshay Kumar|   30|
|         Om Puri|   30|
|       Yuki Kaji|   29|
|Amitabh Bachchan|   28|
+----------------+-----+
only showing top 10 rows

